**Twitter Elections Jake**<br>
*Compsci 209 Final Project*

Follow the instructions here for pulling tweets:<br>
https://developer.twitter.com/en/docs/tweets/search/api-reference/premium-search#DataEndpoint

Tutorial for pulling geography of tweets:<br>
https://stackabuse.com/accessing-the-twitter-api-with-python/

Query Parameter:<br>
https://developer.twitter.com/en/docs/tweets/search/api-reference/get-search-tweets

----

In [1]:
from twython import Twython
import pandas as pd
import sys

In [2]:
import config_twitter

app_key = config_twitter.api_key
app_secret_key = config_twitter.api_secret_key
access_token = config_twitter.access_token
access_token_secret = config_twitter.access_token_secret

user_agent = "Jake_Schneider"

In [3]:
twython = Twython(app_key, app_secret_key,
                  access_token, access_token_secret)

In [4]:
twython.get_application_rate_limit_status()['resources']['search']

{'/search/tweets': {'limit': 180, 'remaining': 180, 'reset': 1575914010}}

In [5]:
twython.verify_credentials()

{'id': 1293308917,
 'id_str': '1293308917',
 'name': 'Jake Schneider',
 'screen_name': '_jwschneider',
 'location': 'Cambridge, MA',
 'description': 'Economist. Data Scientist. Entrepreneur. \nFounder @SchneiderEconomics. Founder @PolicyViews. MPA/ID Candidate @Harvard @Kennedy_School.',
 'url': 'https://t.co/KiRNKCPigk',
 'entities': {'url': {'urls': [{'url': 'https://t.co/KiRNKCPigk',
     'expanded_url': 'http://www.policyviews.com',
     'display_url': 'policyviews.com',
     'indices': [0, 23]}]},
  'description': {'urls': []}},
 'protected': False,
 'followers_count': 59,
 'friends_count': 171,
 'listed_count': 0,
 'created_at': 'Sun Mar 24 02:33:01 +0000 2013',
 'favourites_count': 48,
 'utc_offset': None,
 'time_zone': None,
 'geo_enabled': False,
 'verified': False,
 'statuses_count': 59,
 'lang': None,
 'status': {'created_at': 'Sun Nov 24 15:52:33 +0000 2019',
  'id': 1198630493694169089,
  'id_str': '1198630493694169089',
  'text': "@BarackObama Recently #Harvard's student 

----

**Search for Conservative Tweets**

In [6]:
# Create query
query = {'q': 'conservative',
        #'geocode': '37.0902, -95.7129, 1500mi',
        'result_type': 'popular',
        'count': 1000,
        'lang': 'en',
#        'until': "2018-11-06" 
        }

print(query)

{'q': 'conservative', 'result_type': 'popular', 'count': 1000, 'lang': 'en'}


In [7]:
twython.get_application_rate_limit_status()['resources']['search']

{'/search/tweets': {'limit': 180, 'remaining': 180, 'reset': 1575914015}}

In [8]:
# Search tweets
dict_ = {'user': [], 'date': [], 'text': [], 'favorite_count': [], 'user_loc': []}
for status in twython.search(**query)['statuses']:
    dict_['user'].append(status['user']['screen_name'])
    dict_['date'].append(status['created_at'])
    dict_['text'].append(status['text'])
    dict_['favorite_count'].append(status['favorite_count'])
    dict_['user_loc'].append(status['user']['location'])


    
print(len(dict_["user"]))

15


In [9]:
# Structure data in a pandas DataFrame for easier manipulation
df = pd.DataFrame(dict_)
df.sort_values(by='favorite_count', inplace=True, ascending=False)
df.head(100)

,user,date,text,favorite_count,user_loc
2,mrjamesob,Sun Dec 08 23:18:32 +0000 2019,It’s not hard. Vote Conservative if you think ...,20522,London
0,charliekirk11,Sun Dec 08 13:20:34 +0000 2019,This is your daily reminder that Obama weaponi...,20391,"Phoenix, AZ"
4,jeremycorbyn,Mon Dec 09 12:32:41 +0000 2019,The Conservative slogan “Get Brexit done” is a...,16696,UK
3,JennaEllisEsq,Sun Dec 08 17:41:22 +0000 2019,The Dems are desperately searching for grounds...,14208,"Washington, DC"
5,Aiannucci,Mon Dec 09 10:29:47 +0000 2019,If you normally vote Conservative I’d like to ...,9432,
1,dbongino,Mon Dec 09 01:47:22 +0000 2019,Ditch the Drudge Report and follow the real so...,8862,"Florida, USA"
10,BorisJohnson,Sun Dec 08 18:23:00 +0000 2019,Let's end the chaos and uncertainty. Let's get...,4522,United Kingdom
7,BorisJohnson,Sun Dec 08 09:30:00 +0000 2019,"Under my leadership, a majority Conservative g...",3926,United Kingdom
9,BorisJohnson,Mon Dec 09 09:39:00 +0000 2019,We will get Brexit done and get Parliament wor...,3899,United Kingdom
11,BorisJohnson,Sun Dec 08 10:05:00 +0000 2019,We must respect the result of the EU Referendu...,2951,United Kingdom


In [10]:
# Geographic Heat Map

from geopy.geocoders import Nominatim
import gmplot

geolocator = Nominatim(user_agent = user_agent)

# Go through all tweets and add locations to 'coordinates' dictionary
coordinates = {'latitude': [], 'longitude': []}
for count, user_loc in enumerate(df.user_loc):
    try:
        location = geolocator.geocode(user_loc)
        
        # If coordinates are found for location
        if location:
            coordinates['latitude'].append(location.latitude)
            coordinates['longitude'].append(location.longitude)
            
    # If too many connection requests
    except:
        pass
    
# Instantiate and center a GoogleMapPlotter object to show our map
gmap = gmplot.GoogleMapPlotter(30, 0, 3)

# Insert points on the map passing a list of latitudes and longitudes
gmap.heatmap(coordinates['latitude'], coordinates['longitude'], radius=20)

# Save the map to html file
gmap.draw("/Users/jschneids13/Desktop/Harvard Kennedy School MPA-ID/Semesters/Semester 3, Fall 2019/COMPSCI 109 Data Science/Final Project/CS109a-Group-81-Final-Project/API/heat_map.html")

----

**Premium Query: Example "Conservative"** <br>
<br>
Resource from Luca Hammer: https://lucahammer.com/2019/11/05/collecting-old-tweets-with-the-twitter-premium-api-and-python/ <br>
<br>
Search Query Parameters from Fresh Van Roote: https://freshvanroot.com/blog/2019/twitter-search-guide-by-luca/

In [11]:
!pip install searchtweets

In [12]:
# Load OAUTH Bearer Token Credentials

from searchtweets import load_credentials

premium_search_args = load_credentials("twitter_keys_full_archive.yaml",
                                       yaml_key="search_tweets_api",
                                       env_overwrite=False)
#print(premium_search_args)

/opt/anaconda3/lib/python3.7/site-packages/searchtweets/credentials.py:34: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  search_creds = yaml.load(f)[yaml_key]
Grabbing bearer token from OAUTH


In [13]:
# Create General Payload Rule

from searchtweets import gen_rule_payload

rule = gen_rule_payload("Conservative", 
                        results_per_call=100,
                        from_date="2018-08-06",
                        to_date="2018-11-06"
                        #next_results=1
                       )

In [14]:
# Create Result Stream
# Pagination: https://github.com/twitterdev/search-tweets-python/issues/47
# Try max_pages function

from searchtweets import ResultStream

rs = ResultStream(rule_payload=rule,
                  max_results=100, #twitter allows 900 max per call
                  #max_pages=100,
                  #max_id=1001,
                  **premium_search_args)
print(rs)

ResultStream: 
	{
    "username": null,
    "endpoint": "https://api.twitter.com/1.1/tweets/search/fullarchive/elections.json",
    "rule_payload": {
        "query": "Conservative",
        "maxResults": 100,
        "toDate": "201811060000",
        "fromDate": "201808060000"
    },
    "tweetify": true,
    "max_results": 100
}


In [15]:
## Collect Tweets Using 'collect_results'
#from searchtweets import collect_results
#
#results_list = collect_results(rule,
#                              max_results=500,
#                              result_stream_args= premium_search_args)

In [16]:
## View length results_list
#
#print(len(results_list))

In [17]:
## View results_list
#
#results_list[0:1]

In [18]:
# Collect the Tweets

import json

with open('twitter_premium_api_demo_conservative.jsonl', 'a', encoding='utf-8') as f:
    n = 0
    for tweet in rs.stream():
        n += 1
        if n % 10 == 0:
            print('{0}: {1}'.format(str(n), tweet['created_at']))
        json.dump(tweet, f)
        f.write('\n')
print('done')

retrying request; current status code: 429
retrying request; current status code: 429
retrying request; current status code: 429
HTTP Error code: 429: Request exceeds account’s current package request limits. Please upgrade your package and retry or contact Twitter about enterprise access.
Rule payload: {'query': 'Conservative', 'maxResults': 100, 'toDate': '201811060000', 'fromDate': '201808060000'}


HTTPError: 

In [19]:
# Create Date Function for For Loop
from datetime import timedelta, date, datetime, time

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

dates = []
start_date = datetime(2018, 10, 6, 15, 0)
end_date = datetime(2018, 11, 7, 15, 0)
for single_date in daterange(start_date, end_date):
    #print(single_date.strftime("%Y-%m-%d"))
    dates.append(str(single_date))

print(dates)


['2018-10-06 15:00:00', '2018-10-07 15:00:00', '2018-10-08 15:00:00', '2018-10-09 15:00:00', '2018-10-10 15:00:00', '2018-10-11 15:00:00', '2018-10-12 15:00:00', '2018-10-13 15:00:00', '2018-10-14 15:00:00', '2018-10-15 15:00:00', '2018-10-16 15:00:00', '2018-10-17 15:00:00', '2018-10-18 15:00:00', '2018-10-19 15:00:00', '2018-10-20 15:00:00', '2018-10-21 15:00:00', '2018-10-22 15:00:00', '2018-10-23 15:00:00', '2018-10-24 15:00:00', '2018-10-25 15:00:00', '2018-10-26 15:00:00', '2018-10-27 15:00:00', '2018-10-28 15:00:00', '2018-10-29 15:00:00', '2018-10-30 15:00:00', '2018-10-31 15:00:00', '2018-11-01 15:00:00', '2018-11-02 15:00:00', '2018-11-03 15:00:00', '2018-11-04 15:00:00', '2018-11-05 15:00:00', '2018-11-06 15:00:00']


In [20]:
# Create For Loop To Pull Max Tweets

premium_search_args = load_credentials("twitter_keys_full_archive.yaml",
                                       yaml_key="search_tweets_api",
                                       env_overwrite=False)

for date in dates:
    #date = datetime.strptime(str(day), "%Y-%m-%d")
    print(date)
    rule = gen_rule_payload("Conservative", 
                        results_per_call=100,
                        from_date= date[:-8]+str("12:00"), # 12:00pm
                        to_date= date[:-3] # 3:00pm
                        #next_results=1
                       )
    print(rule)
    
    rs = ResultStream(rule_payload=rule,
                      max_results=500, #twitter allows 900 max per call
                      #max_id=1001,
                      **premium_search_args)
    
    with open('twitter_conservative.jsonl', 'a', encoding='utf-8') as f:
        n = 0
        for tweet in rs.stream():
            n += 1
            if n % 10 == 0:
                print('{0}: {1}'.format(str(n), tweet['created_at']))
                json.dump(tweet, f)
                f.write('\n')
    print('done')

Grabbing bearer token from OAUTH


2018-10-06 15:00:00
{"query": "Conservative", "maxResults": 100, "toDate": "201810061500", "fromDate": "201810061200"}
2018-10-07 15:00:00
{"query": "Conservative", "maxResults": 100, "toDate": "201810071500", "fromDate": "201810071200"}
2018-10-08 15:00:00
{"query": "Conservative", "maxResults": 100, "toDate": "201810081500", "fromDate": "201810081200"}
2018-10-09 15:00:00
{"query": "Conservative", "maxResults": 100, "toDate": "201810091500", "fromDate": "201810091200"}
2018-10-10 15:00:00
{"query": "Conservative", "maxResults": 100, "toDate": "201810101500", "fromDate": "201810101200"}
2018-10-11 15:00:00
{"query": "Conservative", "maxResults": 100, "toDate": "201810111500", "fromDate": "201810111200"}
2018-10-12 15:00:00
{"query": "Conservative", "maxResults": 100, "toDate": "201810121500", "fromDate": "201810121200"}
2018-10-13 15:00:00
{"query": "Conservative", "maxResults": 100, "toDate": "201810131500", "fromDate": "201810131200"}
2018-10-14 15:00:00
{"query": "Conservative", "m

In [ ]:
# Install requests_cache
!pip install requests-cache

In [ ]:
# Need to add cacheing to avoid rate limiting

import requests_cache
requests_cache.install_cache()

In [ ]:
# Perform Cacheing
## This code needs to be changed to twitter
## Possible link for pagination: https://stackoverflow.com/questions/14265082/query-regarding-pagination-in-tweepy-get-followers-of-a-particular-twitter-use
## Here's another: https://bhaskarvk.github.io/2015/01/how-to-use-twitters-search-rest-api-most-effectively./
## Pages in tweepy: https://medium.com/@dmitryrastorguev/sentiment-analysis-of-twitter-timelines-61c73eeacedf

#import time
#from IPython.core.display import clear_output
#
#responses = []
#
#page = 1
#total_pages = 99999 # this is just a dummy number so the loop starts
#
#while page <= total_pages:
#    payload = {
#        'method': 'chart.gettopartists',
#        'limit': 500,
#        'page': page
#    }
#
#    # print some output so we can see the status
#    print("Requesting page {}/{}".format(page, total_pages))
#    # clear the output to make things neater
#    clear_output(wait = True)
#
#    # make the API call
#    response = lastfm_get(payload)
#
#    # if we get an error, print the response and halt the loop
#    if response.status_code != 200:
#        print(response.text)
#        break
#
#    # extract pagination info
#    page = int(response.json()['artists']['@attr']['page'])
#    total_pages = int(response.json()['artists']['@attr']['totalPages'])
#
#    # append response
#    responses.append(response)
#
#    # if it's not a cached result, sleep
#    if not getattr(response, 'from_cache', False):
#        time.sleep(0.25)
#
#    # increment the page number
#    page += 1

In [21]:
# Load Json File and Convert to PD Dataframe

tweets_conservative_df = pd.read_json('twitter_premium_api_demo_conservative.jsonl', lines=True)
tweets_conservative_df.head()

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,lang,matching_rules,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink,display_text_range,extended_tweet,possibly_sensitive,extended_entities
0,2019-12-06 21:04:29+00:00,1203057649039937537,1203057649039937536,RT @Conservatives: TRIPLE TAX LOCK: We will no...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,en,[{'tag': None}],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-12-06 21:04:29+00:00,1203057648486232064,1203057648486232064,RT @RobBurl: There’s a certain irony to this. ...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,en,[{'tag': None}],1.202881e+18,1.202881e+18,{'created_at': 'Fri Dec 06 09:20:40 +0000 2019...,"{'url': 'https://t.co/UiQ2iVh299', 'expanded':...",NaN,NaN,NaN,NaN
2,2019-12-06 21:04:29+00:00,1203057647701901313,1203057647701901312,RT @SHKMEP: I am a Conservative proud Brit\nI ...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,en,[{'tag': None}],1.202955e+18,1.202955e+18,{'created_at': 'Fri Dec 06 14:18:15 +0000 2019...,"{'url': 'https://t.co/H08moq1Of1', 'expanded':...",NaN,NaN,NaN,NaN
3,2019-12-06 21:04:29+00:00,1203057645965340672,1203057645965340672,RT @imajsaclaimant: Disabled people put on a s...,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,NaN,NaN,NaN,...,en,[{'tag': None}],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-12-06 21:04:28+00:00,1203057644925272071,1203057644925272064,RT @BorisJohnson: If you work hard to provide ...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,en,[{'tag': None}],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
# View Df Collumns
tweets_conservative_df.columns

Index(['created_at', 'id', 'id_str', 'text', 'source', 'truncated',
       'in_reply_to_status_id', 'in_reply_to_status_id_str',
       'in_reply_to_user_id', 'in_reply_to_user_id_str',
       'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place',
       'contributors', 'retweeted_status', 'is_quote_status', 'quote_count',
       'reply_count', 'retweet_count', 'favorite_count', 'entities',
       'favorited', 'retweeted', 'filter_level', 'lang', 'matching_rules',
       'quoted_status_id', 'quoted_status_id_str', 'quoted_status',
       'quoted_status_permalink', 'display_text_range', 'extended_tweet',
       'possibly_sensitive', 'extended_entities'],
      dtype='object')

In [23]:
# View Location Data through "GEO"
tweets_conservative_df[["geo", "coordinates", "place", "user"]].head()

,geo,coordinates,place,user
0,None,None,None,"{'id': 176217968, 'id_str': '176217968', 'name..."
1,None,None,None,"{'id': 2808839789, 'id_str': '2808839789', 'na..."
2,None,None,None,"{'id': 2175096381, 'id_str': '2175096381', 'na..."
3,None,None,None,"{'id': 147322918, 'id_str': '147322918', 'name..."
4,None,None,None,"{'id': 422238927, 'id_str': '422238927', 'name..."


In [24]:
# View User Information through "User"
print(len(tweets_conservative_df))

for loc in range(0,len(tweets_conservative_df)):
    print(tweets_conservative_df["user"][loc]["location"])

6798
None
Manchester, England
None
Wolverhampton
None
Dartford, Kent
Neath, Wales
None
None
Liverpool, England
None
None
None
None
Portland, OR
New Jersey, USA
London
Cumbria, England
Nebraska, USA
Toronto, ON Canada 🍁
None
North West, England
London
None
Canada     I ❤️ 🍁 Oil & Gas
Manchester, England
Godalming
England
The place you'll never know
None
Boston, MA 🇺🇸
The Moon
London, England
None
London/Manchester
None
England, United Kingdom
United Kingdom
United Kingdom
Scotland, United Kingdom
North Essex
Liverpool
London, England
Hackney
Espanja
England, United Kingdom
None
England, United Kingdom
None
UK
None
Cotswolds & SW France
United Kingdom
Germany
United Kingdom
Keighley, England
None
United Kingdom
North Tyneside
Cheshire
None
Liam💖
None
Buttstädt, Deutschland
Herefordshire, UK
North West, England
England, United Kingdom
scotland | she/her
Liverpool, England
South East, England
Cambridge
Mordor / Bristol
Los Angeles, CA, USA
Bristol
None
Belfast
None
Finchley
Manchester, Eng

Somewhere in the universe 
None
None
None
None
None
Toronto,Ontario Canada
Norwich, CT
Ecclesia Catholica
None
None
None
None
None
Lives in Exceptional America
Texas Hill Country
Massachusetts, USA
None
60047
Orange Park, FL
Pennsylvania, USA
Saskatchewan, Canada
None
“Not tired of winning yet”
Connecticut
IVotedObamaAndIDidntLikeIt
Vancouver, WA
Tennessee, USA
Texas, the best state
United States
None
Toronto,Ontario Canada
Division No.  6, Saskatchewan
boston

None
None
Washington, DC
Columbus, OH
boston

Various, USA
Blue In A Red state
Beachwood, OH
United States
Yorkshire UK
TX
Liverpool, NY
Pennsylvania
Southern Illinois
None
Washington, DC
N.E TN, USA
Ecclesia Catholica
South Florida, USA
Richmond, VA
Seattle
USA
WithLove StandingWithIsrael
United States
cleveland ohio
Newcastle upon Tyne
Newcastle upon Tyne
DC via NC via GA
None
None
United States
Fort Payne, AL
Wellington City, New Zealand
Oklahoma, USA
chicago
Los Angeles
None
Winnipeg, Manitoba
None
Monroe, MI
Massachusetts, 

Warren, MI
Lambeth, London
None
NY
MAGALAND
MAGALAND
Geordie diaspora
Staten Island, NY
San Antonio, Texas
Trump USA 
None
Milwaukee, WI
Milwaukee, WI
GodsCountry
United States
4300 Montana @ Gateway 54
None
None
Mayagüez, Puerto Rico
Texas, USA
United States
London
United States
None
South Range, Wi. USA
Cental CA (where it's red)
I FOLLOW BACK
Seattle-ish
Santa Barbara, CA
atlanta
Colorado, USA
Outer Space
uk
None
Canada, British Columbia, Port Coquitlam
Frisco, TX
None
Newberg, OR
None
Laredo
San Marvelous 
Middle America
Derbyshire
None
United Kingdom
Illinois, USA
United States
United States
USA
United Kingdom
Nobodys business
Bronzebeard-US
United States
New Jersey, USA
North West, England
None
Georgia, USA
Indiana
NY, NJ, PA, CT Proudly Blocked by USNJack
Mississippi, USA
None
L.A. girl now in Ohio
United States
United Kingdom
None
None
 WNY USA
None
None
United States
Devon. EX15 3BS
United States
Chicago, IL
None
Cornwall
 England
None
Frisco, TX
None
Texas, USA
South Jacksonv

Somewhere in the universe 
None
None
None
None
None
Toronto,Ontario Canada
Norwich, CT
Ecclesia Catholica
None
None
None
None
None
Lives in Exceptional America
Texas Hill Country
Massachusetts, USA
None
60047
Orange Park, FL
Pennsylvania, USA
Saskatchewan, Canada
None
“Not tired of winning yet”
Connecticut
IVotedObamaAndIDidntLikeIt
Vancouver, WA
Tennessee, USA
Texas, the best state
United States
None
Toronto,Ontario Canada
Division No.  6, Saskatchewan
boston

None
None
Washington, DC
Columbus, OH
boston

Various, USA
Blue In A Red state
Beachwood, OH
United States
Yorkshire UK
TX
Liverpool, NY
Pennsylvania
Southern Illinois
None
Washington, DC
N.E TN, USA
Ecclesia Catholica
South Florida, USA
Richmond, VA
Seattle
USA
WithLove StandingWithIsrael
United States
cleveland ohio
Newcastle upon Tyne
Newcastle upon Tyne
DC via NC via GA
None
None
United States
Fort Payne, AL
Wellington City, New Zealand
Oklahoma, USA
chicago
Los Angeles
None
Winnipeg, Manitoba
None
Monroe, MI
Massachusetts, 

United States
Missouri, USA
Canada
Canada
None
Oklahoma, USA
a slave at Panera Bread
Glasgow, Scotland
LI, NY
None
Texas, USA
Planet Earth 
Planet Earth 
United States
Plymouth Township, MI
None
Northern Virginia
hell 
California
she/her
Upstate, NY
Davie, FL🇺🇸 | Calgary 🇨🇦
Birmingham UK
Washington, USA
None
Kalamazoo, MI
None
Richmond, VA
Chicago, IL
None
🇺🇸N. Carolina🇺🇸
Hollywood, Los Angeles
None
None
Minas Gerais, Brasil
None
🕊OneNationUnderGod🇺🇸🦅🇺🇸
Elkhart, IN
None
Coral Gables, FL
South Carolina, USA
None
New York, USA
None
None
WV
Perris, CA
Orlando, FL
Edmonton, Alberta
Rio de Janeiro, Brasil
N Yorks
Arizona
West Palm Beach, FL
United Kingdom
canada
Canada
England
United States
Evansville, IN
None
Sunny Southern California
None
in the present
None
None
None
Adelaide South Australia
None
None
California, USA
Trump area
Smyrna, TN
Erie, PA
None
Ruskin,Florida 
Virginia, USA
Atlanta, GA
El Paso, TX
Alaska, USA
NC & Ohio
in a dream far, far away
Florida, USA
Florida, USA
None
Ann A

---

**Streamer**

In [ ]:
from twython import TwythonStreamer
import csv

# Filter out unwanted data
def process_tweet(tweet):
    d = {}
    d['hashtags'] = [hashtag['text'] for hashtag in tweet['entities']['hashtags']]
    d['text'] = tweet['text']
    d['user'] = tweet['user']['screen_name']
    d['user_loc'] = tweet['user']['location']
    return d
    
    
# Create a class that inherits TwythonStreamer
class MyStreamer(TwythonStreamer):     

    # Received data
    def on_success(self, data):

        # Only collect tweets in English
        if data['lang'] == 'en':
            tweet_data = process_tweet(data)
            print(tweet_data)
#            self.save_to_csv(tweet_data)
            return tweet_data

    # Problem with the API
    def on_error(self, status_code, data):
        print(status_code, data)
        self.disconnect()
        
    # Save each tweet to csv file
#    def save_to_csv(self, tweet):
#        with open(r'saved_tweets_conservative.csv', 'a') as file:
#            writer = csv.writer(file)
#            writer.writerow(list(tweet.values()))

In [ ]:
## Instantiate from our streaming class
#stream = MyStreamer(app_key, app_secret_key,
#                  access_token, access_token_secret)
## Start the stream
#tweets = stream.statuses.filter(track='conservative')